# The Irish Longitudinal Study on Ageing (TILDA)
1. The age distribution of the Irish population is undergoing a dramatic change at present and this trend is predicted to continue into the future. People are living longer, and older persons represent a larger proportion of the population.
2.  a large prospective cohort study of ageing, which includes an assessment of the social, economic and health circumstances of community-resident older people living in Ireland.
3. Begin in Oct 2009 and compelted in Feb 2011 
4. A multi-stage probability sample of addresses are selected across the Ireland
5. All household residents aged >=50 years were eligible to participate in the study
6. 

## Aim
(i) the description of the current economic, social and health circumstances of older people living in Ireland and the creation of a longitudinal database with a combination of social, economic and health data to allow the monitoring of
changes in health and well-being over time and 
(ii) the exploration of the complex interrelationships between numerous potential risk factors and protective factors and their impact on longevity and healthy ageing including for example: 
(a) the role of early childhood experiences on
physical and mental health in later life;
(b) the relationship between physical activity,
obesity and frailty;
(c) the influence of mood disturbances including anxiety and depression on cognitive
and physical health; and
(d) the effect of cognitive impairment on
financial decisions around retirement.

## Data collection
(i) A computer-assisted personal interview (CAPI) by trained social interviewers in participants' own homes.

(ii) A self-completion questionnair(SCQ) in participants' own time

(iii) A comprehensive health assessment in a dedicated health centre, or a modified version delivered in the participant’s home.A notable change to the health assessment was the addition of the magnetic resonance imaging (MRI) brain sub-study in Wave 3.



TILDA has rich data on neurocognitive function,mental health,cardiovascular function, kidney function, locomotion, falls, fear of falling, vision, socioeconomic status and social circumstances, obtained during multiple waves of data collection.

# TILDA pipeline


## Fieldmap processing

1. Determining the type of the images from scanner [fslinfo], only ANALYZE and NIFTI format are accepted.

```shell
dcm2niix . -o analysis DICOM
```
2. Getting the magnitude image (A complex image can be converted into its phase and magnitude components by using [fslcomplex]) and Check that the magnitude image and the phase/fieldmap images have the same resolution.

```shell
fslmaths fieldmap_mag1 -add 3.14 fieldmap_magshift1  
fslmaths fieldmap_mag2 -add 3.14 fieldmap_magshift2
```

3. Brain extraction to fieldmaps to get a tight brain mask which must exclude all non-brain voxels and any voxels with only a small partial volume contribution. The reason for this is that these areas are normally very noisy in the phase image 

```shell
bet fieldmap_magshift2 fieldmap_mag_brain -f 0.6 -g -0.2
```

4. Getting (warpped) phase in radians, because it can be converted into Hz or Tesla.

5. Unwarpping the phase images within the brain mask, i.e. use PRELUDE to do the required phase unwrapping

```shell
prelude -a fieldmap_magshift1 -p fieldmap_ph1 -o fieldmap_ph1_unwrapped -m fieldmap_mag_brain.nii.gz
prelude -a fieldmap_magshift2 -p fieldmap_ph2 -o fieldmap_ph2_unwrapped -m fieldmap_mag_brain.nii.gz
```
6. Getting fieldmap in radians

```shell
fslmaths fieldmap_ph1_unwrapped -sub fieldmap_ph2_unwrapped -mul 1000 -div 5.32 fieldmap_rads -odt float
```

7. Regularising the fieldmap. Fieldmaps can often be noisy or be contaminated around the edges of the brain. To correct for this you can regularise the fieldmap using [fugue]. (fugue performs unwarping of an EPI image based on fieldmap data).
    Some regularisation options are - Gaussian smoothing, despiking and median filtering. Examples of these (in order) are:
```shell
    fugue --loadfmap=fieldmap_rads -s 1 --savefmap=fieldmap_rads
    fugue --loadfmap=fieldmap_rads --despike --savefmap=fieldmap_rads
    fugue --loadfmap=fieldmap_rads -m --savefmap=fieldmap_rads
```    
    Any combination of these regularisations can be performed.



## T1 processing
1. [fsl_anat]
    ```shell
    fsl_anat -i T1
    ```

    * reorient the images to the standard (MNI) orientation [fslreorient2std]
    * automatically crop the image [robustfov]
    * bias-field correction (RF/B1-inhomogeneity-correction) [FAST]
    * registration to standard space (linear and non-linear) [FLIRT and FNIRT]
    * brain-extraction [FNIRT-based or BET]
    * tissue-type segmentation [FAST]
    * subcortical structure segmentation [FIRST]
    
## ASL processing
1. Perform pairwise substraction of control-tag and get perfusion weighted image [asl_file]
    ```shell
    asl_file --data=asldata --ntis=1 --iaf=ctb --ibf=tis --diff --mean=asldiffdata_mean
    ```
2. Motion correction [mcflirt] (mcflirt is an intra-modal motion correction tool designed for use on fMRI time series and based on optimization and registration techniques used in FLIRT) 
    ```shell
    mcflirt -in asldata -out asldata_mc
    ```
3. Quantify CBF from ASL data, including kinetic-model inversion, absolute quantification via a calibration image and registration of the data [oxford_asl]
    
```shell
    oxford_asl -i asldata_mc -o oxasl --spatial --iaf=ctb --ibf=tis --tis 3.6 --casl --bolus 1.8 -c aslcalib --tr 10  --fslanat=T1.anat --te=9 --slicedt=0.03
```

    
    oxford_asl without distortion correction:
    
    -i asldata_mc
    -o oxasl
    --spatial         #Use adaptive spatial smoothing on perfusion 
    --iaf=ctb         #input ASl format: diff,tc,ct
    --ibf=tis         #input block format (for multi-TI): rpt,tis 
    --tis=3.6         #comma separated list of inversion times, e.g. --tis 0.2,0.4,0.6
    --casl            #ASL acquisition is  pseudo cASL (pcASL) rather than pASL
    --bolus 1.8       #Bolus duration 
    -c aslcalib       #M0 calibration image (proton density or mean control image)
    --tr 10           #TR of calibration data 
    --fslanat=T1.anat #An fsl_anat directory from structural image
    --te=9            #Echo time for the readout 
    --slicedt=0.03    #Timing difference between slices
      
```shell
    oxford_asl -i asldata_mc -o oxasl_distcorr --spatial --iaf=ctb --ibf=tis --tis 3.6 --casl --bolus 1.8 -c aslcalib --tr 10 --fslanat=T1.anat --te=9 --slicedt=0.03 --fmap=fieldmap_rads --fmapmag=fieldmap_magshift2 --fmapmagbrain=fieldmap_mag_brain --echospacing=0.0005 --pedir=-y
```   

    oxford_asl with distortion correction:

    -i asldata_mc
    -o oxasl_distcorr
    --spatial
    --iaf=ctb
    --ibf=tis
    --tis=3.6
    --casl
    --bolus 1.8
    -c aslcalib
    --tr 10
    --fslanat=T1.anat
    --te=9
    --slicedt=0.03
    --fmap=fieldmap_rads                #fieldmap image (in rad/s)
    --fmapmag=fieldmap_magshift2        #fieldmap magnitude image - wholehead extracted
    --fmapmagbrain=fieldmap_mag_brain   #fieldmap magnitude image - brain extracted
    --echospacing=0.0005                #Effective EPI echo spacing (sometimes called dwell time) - in seconds
    --pedir=-y                          #Phase encoding direction
    


    

## Data labels

### 266 subjects are suspected of low quality out of 587 subjects according to comments.


### 34 labeled as outliers out of 558 subjects according to ```TILDA_ASL_Exclusions_1_10_21.xlsx```.
* 26 labeled with Poor labeling
* 6 labeled with Dealyed Arrival
* 7 labeled with Motion
* 1 labeled with Low Contrast
* 1 labeled with Poor Signal



# Quality Control Metrics

## ASL 
1. GM & WM Perfusion 
2. Spatial Coefficient of Variance(SCoV)
3. Coefficient of joint variance between GM & WM
4. GM & WM Perfsion on territories

## Registration
1. Sum of squared differences
2. Cross-Correlation
3. Pearson Correlation Coefficient
4. Mutual Information
5. Normalized Mutual Information
6. Correlation ratio


# BRC pipeline

## T1
* Cropping(optional) [robustfov] 
* Brain extraction [BET]
* Reduce FOV->T1 [standard_space_roi]
* Register to std(linear) [FLIRT]
* Register to std(non-linear) [FNIRT]
* Brain extraction->T1_brain_mask
* Defaceing
* Bias-field correction->T1_brain_bias [FAST]
* Tissue segmentation->pve
* Subcortical segmentation [run_first_all] 

## ASL
1. Register ASL to T1 [FLIRT]
2. Partial Volume Correction(Liang et al, DOI: 10.1002/mrm.24279)
3. Register ASL to standard space from T1